# Basig usage of qmrExchange

### Import required libraries

In [13]:
from source.qmr_exchange import Exchange
from source.qmr_exchange import Exchange, Simulator
from source.agents import RandomMarketTaker, NaiveMarketMaker
from datetime import datetime

### Declare basic parameters for the simulation
qmrExchange allows for simulating multiple tickers at once, for statistical arbitrage and high-frequency-trading simulations. In the present case, we simulate 1 week worth of 1 minute data (24/7 trading).


In [14]:
from_date = datetime(2022,1,1)
to_date = datetime(2022,1,15)
time_interval = 'minute'
tickers = ['XYZ']

### Instantiate a Simulator

In [15]:
sim = Simulator(from_date, to_date,time_interval)
sim.exchange.create_asset(tickers[0])

### Add trading agents
- We add a naive market maker that creates both buy and sell orders in each period. It quotes buy and sell prices based on the last traded price and the specified spread percentage.
- We add a market taker that randomly buys and sells (based on the defined probabilities) on each period by means of market ordes (hence the word 'taker').

In [16]:

mm = NaiveMarketMaker(name='market_maker', tickers=tickers, aum=1_000, spread_pct=0.005, qty_per_order=4)
sim.add_agent(mm)

mt = RandomMarketTaker(name='market_taker', tickers=tickers, aum=1_000, prob_buy=.2, prob_sell=.2, qty_per_order=1,seed=42)
sim.add_agent(mt)

### Run the simulation

In [17]:
sim.run()

Retrieve all executed trades of our simulation

In [32]:

print(sim.trades.head(20).to_markdown())

| dt                  | ticker   |   qty |   price | buyer        | seller       |
|:--------------------|:---------|------:|--------:|:-------------|:-------------|
| 2022-01-01 00:00:00 | XYZ      |     1 |  100    | init_seed    | init_seed    |
| 2022-01-01 00:04:00 | XYZ      |     1 |  100.25 | market_taker | market_maker |
| 2022-01-01 00:10:00 | XYZ      |     1 |  100    | market_maker | market_taker |
| 2022-01-01 00:11:00 | XYZ      |     0 |   99.75 | market_maker | market_taker |
| 2022-01-01 00:13:00 | XYZ      |     0 |   99.5  | market_maker | market_taker |
| 2022-01-01 00:14:00 | XYZ      |     0 |   99.25 | market_maker | market_taker |
| 2022-01-01 00:15:00 | XYZ      |     1 |   99.5  | market_taker | market_maker |
| 2022-01-01 00:16:00 | XYZ      |     1 |   99.75 | market_taker | market_maker |
| 2022-01-01 00:18:00 | XYZ      |     2 |   99.5  | market_maker | market_taker |
| 2022-01-01 00:19:00 | XYZ      |     0 |   99.25 | market_maker | market_taker |
| 20

Group asset price in fixed 15 Minute OHLCV Bars

In [34]:
df_15min = sim.get_price_bars(ticker=tickers[0],bar_size='15Min')
print(df_15min.head(20).to_markdown())

| dt                  |   open |   high |   low |   close |   volume |
|:--------------------|-------:|-------:|------:|--------:|---------:|
| 2022-01-01 00:00:00 | 100    | 100.25 | 99.25 |   99.25 |        3 |
| 2022-01-01 00:15:00 |  99.5  |  99.75 | 99    |   99.5  |       11 |
| 2022-01-01 00:30:00 |  99.25 |  99.25 | 98.5  |   98.75 |        2 |
| 2022-01-01 00:45:00 |  98.5  |  98.5  | 98    |   98.24 |        2 |
| 2022-01-01 01:00:00 |  97.99 |  98.73 | 97.99 |   98.23 |        9 |
| 2022-01-01 01:15:00 |  98.48 |  99.23 | 98.48 |   99.23 |       11 |
| 2022-01-01 01:30:00 |  99.48 | 100.23 | 99.48 |   99.73 |        9 |
| 2022-01-01 01:45:00 |  99.48 |  99.48 | 98.98 |   98.98 |        2 |
| 2022-01-01 02:00:00 |  99.23 |  99.73 | 98.73 |   99.73 |        9 |
| 2022-01-01 02:15:00 |  99.48 |  99.48 | 98.73 |   98.73 |        5 |
| 2022-01-01 02:30:00 |  98.98 |  99.73 | 98.98 |   99.23 |       10 |
| 2022-01-01 02:45:00 |  98.98 |  99.98 | 98.98 |   99.73 |        8 |
| 2022

Retrieve a dataframe of an agents holding at each period of time

In [36]:
mt_holdings = sim.get_portfolio_history('market_taker')
mm_holdings = sim.get_portfolio_history('market_maker')

print(mt_holdings.head(20).to_markdown())

| dt                  |    XYZ |    cash |     aum |
|:--------------------|-------:|--------:|--------:|
| 2022-01-01 00:00:00 |   0    | 1000    | 1000    |
| 2022-01-01 00:01:00 |   0    | 1000    | 1000    |
| 2022-01-01 00:02:00 |   0    | 1000    | 1000    |
| 2022-01-01 00:03:00 |   0    | 1000    | 1000    |
| 2022-01-01 00:04:00 | 100.25 |  899.75 | 1000    |
| 2022-01-01 00:05:00 | 100.25 |  899.75 | 1000    |
| 2022-01-01 00:06:00 | 100.25 |  899.75 | 1000    |
| 2022-01-01 00:07:00 | 100.25 |  899.75 | 1000    |
| 2022-01-01 00:08:00 | 100.25 |  899.75 | 1000    |
| 2022-01-01 00:09:00 | 100.25 |  899.75 | 1000    |
| 2022-01-01 00:10:00 |   0    |  999.75 |  999.75 |
| 2022-01-01 00:11:00 |   0    |  999.75 |  999.75 |
| 2022-01-01 00:12:00 |   0    |  999.75 |  999.75 |
| 2022-01-01 00:13:00 |   0    |  999.75 |  999.75 |
| 2022-01-01 00:14:00 |   0    |  999.75 |  999.75 |
| 2022-01-01 00:15:00 |  99.5  |  900.25 |  999.75 |
| 2022-01-01 00:16:00 | 199.5  |  800.5  | 100

### Plot the results

In [21]:
from source.helpers import plot_bars
df_15min = sim.get_price_bars(ticker=tickers[0], bar_size='15Min')
plot_bars(df_15min)

In [28]:
import plotly.express as px
import pandas as pd

df_plot = pd.DataFrame()
df_plot['Market Maker'] = mm_holdings['aum']
df_plot['Market Taker'] = mt_holdings['aum']
fig = px.line(df_plot,labels={'variable':'Agents','value':'Assets Under Management','dt':'Date'})
fig.show()